# 如何在提取时使用参考示例
通过向大语言模型提供参考示例，通常可以提升提取信息的质量。
数据提取旨在从文本及其他非结构化或半结构化格式中生成[结构化表示](/docs/concepts/structured_outputs/)。此类场景中常会用到[工具调用](/docs/concepts/tool_calling)的大语言模型功能。本指南演示如何构建工具调用的少样本示例，以引导数据提取类应用的行为模式。
:::提示本指南重点介绍如何通过工具调用模型来使用示例，但该技术具有普适性，适用于各类场景。同时结合JSON或多或少的基于提示的技术。:::
LangChain 在包含工具调用的LLM消息上实现了[tool-call属性](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html#langchain_core.messages.ai.AIMessage.tool_calls)。更多细节请参阅我们的[工具调用操作指南](/docs/how_to/tool_calling)。为了构建数据提取的参考示例，我们创建了包含以下顺序的聊天记录：
- [HumanMessage](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.human.HumanMessage.html) 包含示例输入；- [AIMessage](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html) 包含示例工具调用的消息；- [工具消息](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.tool.ToolMessage.html) 包含示例工具输出。
LangChain采用这一约定，用于在跨LLM模型供应商的对话中结构化工具调用。
首先我们构建一个提示模板，其中包含这些消息的占位符：

In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked "
            "to extract, return null for the attribute's value.",
        ),
        # ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
        MessagesPlaceholder("examples"),  # <-- EXAMPLES!
        # ↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑
        ("human", "{text}"),
    ]
)

测试模板：

In [2]:
from langchain_core.messages import (
    HumanMessage,
)

prompt.invoke(
    {"text": "this is some text", "examples": [HumanMessage(content="testing 1 2 3")]}
)

ChatPromptValue(messages=[SystemMessage(content="You are an expert extraction algorithm. Only extract relevant information from the text. If you do not know the value of an attribute asked to extract, return null for the attribute's value.", additional_kwargs={}, response_metadata={}), HumanMessage(content='testing 1 2 3', additional_kwargs={}, response_metadata={}), HumanMessage(content='this is some text', additional_kwargs={}, response_metadata={})])

## 定义模式
让我们复用[提取教程](/docs/tutorials/extraction)中的人员模式。

In [3]:
from typing import List, Optional

from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(..., description="The name of the person")
    hair_color: Optional[str] = Field(
        ..., description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(..., description="Height in METERs")


class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

## 定义参考示例
示例可以定义为一组输入-输出对的列表。
每个示例都包含一个示例 `input` 文本和一个示例 `output`，展示应从文本中提取的内容。
:::重要这有点过于深入细节了，可以跳过不看。
示例的格式需要与所使用的API相匹配（例如工具调用或JSON模式等）。
在此，格式化示例将与工具调用API所预期的格式相匹配，因为这是我们正在使用的。:::

In [4]:
import uuid
from typing import Dict, List, TypedDict

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)
from pydantic import BaseModel, Field


class Example(TypedDict):
    """A representation of an example consisting of text input and expected tool calls.

    For extraction, the tool calls are represented as instances of pydantic model.
    """

    input: str  # This is the example text
    tool_calls: List[BaseModel]  # Instances of pydantic model that should be extracted


def tool_example_to_messages(example: Example) -> List[BaseMessage]:
    """Convert an example into a list of messages that can be fed into an LLM.

    This code is an adapter that converts our example to a list of messages
    that can be fed into a chat model.

    The list of messages per example corresponds to:

    1) HumanMessage: contains the content from which content should be extracted.
    2) AIMessage: contains the extracted information from the model
    3) ToolMessage: contains confirmation to the model that the model requested a tool correctly.

    The ToolMessage is required because some of the chat models are hyper-optimized for agents
    rather than for an extraction use case.
    """
    messages: List[BaseMessage] = [HumanMessage(content=example["input"])]
    tool_calls = []
    for tool_call in example["tool_calls"]:
        tool_calls.append(
            {
                "id": str(uuid.uuid4()),
                "args": tool_call.dict(),
                # The name of the function right now corresponds
                # to the name of the pydantic model
                # This is implicit in the API right now,
                # and will be improved over time.
                "name": tool_call.__class__.__name__,
            },
        )
    messages.append(AIMessage(content="", tool_calls=tool_calls))
    tool_outputs = example.get("tool_outputs") or [
        "You have correctly called this tool."
    ] * len(tool_calls)
    for output, tool_call in zip(tool_outputs, tool_calls):
        messages.append(ToolMessage(content=output, tool_call_id=tool_call["id"]))
    return messages

接下来让我们定义示例，然后将它们转换为消息格式。

In [5]:
examples = [
    (
        "The ocean is vast and blue. It's more than 20,000 feet deep. There are many fish in it.",
        Data(people=[]),
    ),
    (
        "Fiona traveled far from France to Spain.",
        Data(people=[Person(name="Fiona", height_in_meters=None, hair_color=None)]),
    ),
]


messages = []

for text, tool_call in examples:
    messages.extend(
        tool_example_to_messages({"input": text, "tool_calls": [tool_call]})
    )

让我们测试一下这个提示

In [6]:
example_prompt = prompt.invoke({"text": "this is some text", "examples": messages})

for message in example_prompt.messages:
    print(f"{message.type}: {message}")

system: content="You are an expert extraction algorithm. Only extract relevant information from the text. If you do not know the value of an attribute asked to extract, return null for the attribute's value." additional_kwargs={} response_metadata={}
human: content="The ocean is vast and blue. It's more than 20,000 feet deep. There are many fish in it." additional_kwargs={} response_metadata={}
ai: content='' additional_kwargs={} response_metadata={} tool_calls=[{'name': 'Data', 'args': {'people': []}, 'id': '240159b1-1405-4107-a07c-3c6b91b3d5b7', 'type': 'tool_call'}]
tool: content='You have correctly called this tool.' tool_call_id='240159b1-1405-4107-a07c-3c6b91b3d5b7'
human: content='Fiona traveled far from France to Spain.' additional_kwargs={} response_metadata={}
ai: content='' additional_kwargs={} response_metadata={} tool_calls=[{'name': 'Data', 'args': {'people': [{'name': 'Fiona', 'hair_color': None, 'height_in_meters': None}]}, 'id': '3fc521e4-d1d2-4c20-bf40-e3d72f1068da', 

## 创建提取器
让我们选择一个LLM。由于我们正在使用工具调用功能，因此需要一个支持工具调用特性的模型。可用的LLM请参阅[此表格](/docs/integrations/chat)。
import ChatModelTabs from "@theme/ChatModelTabs";
<ChatModelTabs
customVarName="llm"overrideParams={{openai: {model: "gpt-4-0125-preview", kwargs: "temperature=0"}}}/>

In [7]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)

按照[提取教程](/docs/tutorials/extraction)的指引，我们使用`.with_structured_output`方法根据所需模式结构化模型输出：

In [8]:
runnable = prompt | llm.with_structured_output(
    schema=Data,
    method="function_calling",
    include_raw=False,
)

## 没有示例 😿
请注意，即便是性能强大的模型也可能在**极其简单**的测试用例上失败！

In [9]:
for _ in range(5):
    text = "The solar system is large, but earth has only 1 moon."
    print(runnable.invoke({"text": text, "examples": []}))

people=[Person(name='earth', hair_color='null', height_in_meters='null')]


people=[Person(name='earth', hair_color='null', height_in_meters='null')]


people=[]


people=[Person(name='earth', hair_color='null', height_in_meters='null')]


people=[]


## 通过示例说明 😻
参考示例有助于修复故障！

In [10]:
for _ in range(5):
    text = "The solar system is large, but earth has only 1 moon."
    print(runnable.invoke({"text": text, "examples": messages}))

people=[]


people=[]


people=[]


people=[]


people=[]


请注意，我们可以在 [Langsmith 追踪记录](https://smith.langchain.com/public/4c436bc2-a1ce-440b-82f5-093947542e40/r) 中将少量示例视为工具调用。
而我们在一组阳性样本上保持了性能表现：

In [11]:
runnable.invoke(
    {
        "text": "My name is Harrison. My hair is black.",
        "examples": messages,
    }
)

Data(people=[Person(name='Harrison', hair_color='black', height_in_meters=None)])